# Running scripts in singularity

* **Difficulty level**: intermediate
* **Time need to lean**: 30 minutes or less
* **Key points**:
  * Using `container='library:xxx'` etc will enable `engine='singularity'`
  * `engine='singularity'` is explicitly needed for docker images.  

Before you run any script using singularity, please check if `singularity` is installed by checking the availability of command `singularity`. Also, it would be helpful for you to read the [sigularity user's guide](https://sylabs.io/guides/3.3/user-guide/index.html) or some online tutorial to understand how singularity works before you try to use singularity in SoS.

Although running singularity images does not need root privilege, building singularity images often requires `sudo` access. SoS provides an option `sudo=True` to the `singularity_build` action (an equivalence to command `singularity build`) to execute the command in sudo mode, but it does not accept interactive input of password. So before running any `singularity_build` action with option `sudo=True`, please run `sudo -i` to enter sudo mode, or add your username as a sudo user without password (google "sudo without password" for instructions). The latter is not safe but can be very convenient for you to prepare singularity images on a personal linux workstation.

## Running a script using singularity

### Options `container` and `engine`

SoS executes scripts with a singularity container by calling command `singularity exec` with appropriate parameters. For example, if you specify a container with a `library:` schema, sos will first pull the image, save it as a local image, and use `singularity exec` to run it:

In [1]:
%run -v1
sh: container='library://alpine:latest'
  cat /etc/os-release

HINT: Pulling image library://alpine:latest to ~/.sos/singularity/library/alpine-latest.sif
NAME="Alpine Linux"
ID=alpine
VERSION_ID=3.9.2
PRETTY_NAME="Alpine Linux v3.9"
HOME_URL="https://alpinelinux.org/"
BUG_REPORT_URL="https://bugs.alpinelinux.org/"


The actual `singularity exec` command executed by SoS can be shown when you execute the script in dryrun mode (with `-n` option). In this mode, SoS would print the interpolated script (if option `expand=True` is set) and the docker command to execute it:

In [2]:
%run -v1 -n
sh: container='library://alpine:latest'
  cat /etc/os-release

HINT: singularity exec  /home/bpeng1/.sos/singularity/library/alpine-latest.sif /bin/sh /home/bpeng1/sos/sos-docs/src/user_guide/tmpurin91a5/singularity_run_32484.sh
cat /etc/os-release



As you can see, the docker command looks similar to

```
singularity exec alpine-latest.sif /bin/sh /path/to/a/temp/script
```

Basically, SoS pulls the image and runs command `singularity exec` to execte the script with `/bin/sh` with the singularity image.

You can also use a docker image with singularity. However, because `docker://` images are by default executed by docker, you will need to specify the use of singularity using parameter `engine='singularity'`:

In [3]:
%run -v1
sh: container='docker://ubuntu', engine='singularity'
  cat /etc/os-release

HINT: Pulling image docker://ubuntu to ~/.sos/singularity/library/ubuntu.sif
NAME="Ubuntu"
VERSION="18.04.3 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.3 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In summary, as listed [here](https://vatlab.github.io/sos-docs/doc/user_guide/sos_actions.html), you can invoke `singularity` with the following combinations of parameters `container` and `engine`:

| `container` | `engine` | execute by | example | comment | 
| -- | -- | -- | -- | -- |
| `filename.simg` | ` ` | singularity | `container='ubuntu.simg'` | |
| `shub://tag` | ` ` | singularity | `container='shub://GodloveD/lolcow'` | Image will be pulled to a local image |
| `library://tag` | ` ` | singularity | `container='library://GodloveD/lolcow'` | Image will be pulled to a local image |
| `name` | `singularity` | singularity | `container='a_dir', engine='singularity'` | treat `name` as singularity image file or directory |
| `docker://tag` | `singularity` | singularity |  `container='docker://godlovdc/lolcow', engine='singularity'`  |  |
| `file://filename` | ` ` | singularity | `container='file://ubuntu.simg'` | |

Simply put, singularity will be used by default with container `shub://`, `file://`, and `filename.simg`, but you will have to specify `engine='singularity'` if you would like to use a docker image or a directory.

### Running script with singularity image

Although singularity accepts `docker://`, `shub://`, and 'library://' container types, SoS always pull the image and build a local `.sif` file before executing it. If the container is used again, the local `.sif` file will be used directly.

For example, because an image has already been downloaded from the above example, SoS will not pull `docker://ubuntu` when using the container again

In [4]:
run: container='docker://ubuntu', engine='singularity'
  ls /

ls /
bin   dev	   etc	 lib	media  opt   root  sbin		srv  tmp  var
boot  environment  home  lib64	mnt    proc  run   singularity	sys  usr


You can use the name of the image directly (e.g. `container='ubuntu.sif'`) if you know the image has been downloaded:

In [5]:
run: container='ubuntu.sif'
  ls /

ls /
bin   dev	   etc	 lib	media  opt   root  sbin		srv  tmp  var
boot  environment  home  lib64	mnt    proc  run   singularity	sys  usr


### Customize singularity image path

By default, downloaded singularity images are saved to the SoS Singularity Library located at `~/.sos/singularity/library`. Environment variable `SOS_SINGULARITY_LIBRARY` can be used to set customized directory, e.g.,

```bash
export SOS_SINGULARITY_LIBRARY=/path/to/sif_folder
```

SoS will use `*.sif` files in `$SOS_SINGULARITY_LIBRARY` if they exist. As a result you can manually place existing image files to `$SOS_SINGULARITY_LIBRARY` to avoid pull images online attempts. This is useful when for example a workflow runs on a compute node without internet access.

### Binding directories (option `bind`)

A very useful feature about singularity is that you can use the container almost as a local command with access to local file system,

In [6]:
run: container='ubuntu.sif'
  wc -l ~/.bashrc

wc -l ~/.bashrc
136 /home/bpeng1/.bashrc


However, singularity only binds current working directory, `/tmp`, and your home directory. Other directories would be from within the image, or appear to be missing even if they exist on the host file system. For example, the following command lists `/usr/local` inside the image:

In [7]:
run: container='ubuntu.sif'
  ls /usr/local

ls /usr/local
bin  etc  games  include  lib  man  sbin  share  src


and `/usr/local/var` would appear to be missing

In [8]:
%env --expect-error
run: container='ubuntu.sif'
  ls /usr/local/var

ls /usr/local/var
ls: cannot access '/usr/local/var': No such file or directory


ExecuteError: [0]: Executing script in Singularity returns an error (exitcode=2).
The script has been saved to /home/bpeng1/sos/sos-docs/src/user_guide/.sos/singularity_run_32376.sh. To reproduce the error please run:
``singularity exec  /home/bpeng1/.sos/singularity/library/ubuntu.sif /bin/bash -ev /home/bpeng1/sos/sos-docs/src/user_guide/.sos/singularity_run_32376.sh``

To allow singularity to see more directories, you can add one or more parameters to the `bind` parameter. For example, with `bind='/usr/local'`, the `singularity exec` command lists directory `/usr/local` from the host filesystem:

In [9]:
run: container='ubuntu.sif', bind='/usr/local'
  ls /usr/local

ls /usr/local
bin  etc  games  go  include  lib  libexec  man  sbin  share  src  var


and we can see `/usr/local/var` actually exists

In [10]:
run: container='ubuntu.sif', bind='/usr/local'
  ls /usr/local/var

ls /usr/local/var
singularity


Parameter `bind` accepts `host_dir:img_dir` pairs to mount `host_dir` from host as `img_dir` seen by the image

In [11]:
%run -v1
run: container='ubuntu.sif', bind='/usr/local/var:/myvar'
  ls /myvar

ls /myvar
singularity


### Other `singularity exec` options

## Building singularity images

If you need to pass other options to command `singularity exec`, you can specify them as additional keyword arguments for the `run` (or other) actions. For example,

In [ ]:
run: container='docker://ubuntu', engine='singularity', disable_cache=True
  echo 'Echo'

Note that you need to use format `OPT=True` to specify flag options such as `--nv`, and need to use underscore to specify options with `-` in their names.

### Action `singularity_build` <a id="singularity_build"></a>

Action `singularity_build` calls command `singularity build` with appropriate command line options. The SoS equivalence of the example in the singularity user's guide

```
singularity build lolcow.sif library://sylabsed/examples/lolcow
```
is the `singularity_build` action with options `src` and `dest`:

In [12]:
singularity_build(src='library://sylabsed/examples/lolcow', dest='lolcow.sif', force=True)

INFO:    Starting build...
INFO:    Creating SIF file...
INFO:    Build complete: lolcow.sif


0

You can also use action `singularity_build` to build an image from a singularity definition file. Using an example from the user's guide:

In [13]:
singularity_build: dest='lolcow.sif', sudo=True, force=True
    Bootstrap: docker
    From: ubuntu:16.04
    %post
        apt-get -y update
        apt-get -y install fortune cowsay lolcat
    %environment
        export LC_ALL=C
        export PATH=/usr/games:$PATH
    %runscript
        fortune | cowsay | lolcat 

INFO:    Starting build...
Getting image source signatures
Copying blob 3386e6af03b0 [--------------------------------------] 0b / 0b
Copying blob 49ac0bbe6c8e [--------------------------------------] 0b / 0b
Copying blob d1983a67e104 [--------------------------------------] 0b / 0b
Copying blob 1a0f3a523f04 [--------------------------------------] 0b / 0b
Copying blob 3386e6af03b0 skipped: already exists
Copying blob 49ac0bbe6c8e skipped: already exists
Copying blob d1983a67e104 skipped: already exists
Copying blob 1a0f3a523f04 skipped: already exists
Copying config ea9ba824b3 [====================================] 2.4KiB / 2.4KiB
Copying config ea9ba824b3 done
Writing manifest to image destination
Storing signatures
2019/12/30 11:38:13  info unpack layer: sha256:3386e6af03b043219225367632569465e5ecd47391d1f99a6d265e51bd463a83
2019/12/30 11:38:14  info unpack layer: sha256:49ac0bbe6c8eeb959337b336ceaa5c3bbbae81e316025f9b94ede453540f2377
2019/12/30 11:38:14  info unpack layer: sha256:d

0

Options such as `notest=True` could be add to the action. Note that the content of the definition file is indented for  clarify, but you can include the file as it is (no indentation).